In [1]:
import pandas as pd


# Reading the csv file
df_train = pd.read_csv("./train.csv")
df_train.head(5)

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [2]:
# !pip3 install transformers
# !pip3 install SentencePiece

In [17]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from string import Template
device = "cuda:0" if torch.cuda.is_available() else "cpu"

llm = 'google/flan-t5-small'

model = T5ForConditionalGeneration.from_pretrained(llm).to(device)
tokenizer = T5Tokenizer.from_pretrained(llm)


In [9]:

inputs = tokenizer("The capital of America is : ", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['United States of America']


In [43]:
text  = """
Give below is a question labelled as 'Q' : and 5 possible answers to the question labelled as 'A':,'B':,'C':,'D':,'E':. Your task is to predict the most likely answer to the question. Your output should be either of A,B,C,D,E.

'Q' : $q

'A' : $a
'B' : $b
'C' : $c
'D' : $d
'E' : $e



"""

template = Template(text)

In [44]:
x = template.substitute(q="Q",a = "this",b='test',c='test',d='test',e='test')

In [45]:
def format_dataframe(df):
    
    text = []
    for i in range(len(df)):
        q = df.loc[i,'prompt']
        a = df.loc[i,'A']
        b = df.loc[i,'B']
        c = df.loc[i,'C']
        d = df.loc[i,'D']
        e = df.loc[i,'E']

        text.append (template.substitute(q=q,a = a,b=b,c=c,d=d,e=e))
    
    return text




In [46]:
input = format_dataframe(df_train)

In [47]:
print(input[0])


Give below is a question labelled as 'Q' : and 5 possible answers to the question labelled as 'A':,'B':,'C':,'D':,'E':. Your task is to predict the most likely answer to the question. Your output should be either of A,B,C,D,E.

'Q' : Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?

'A' : MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
'B' : MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
'C' : MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
'D' : MOND is a

In [50]:
inputs = tokenizer(input[0], return_tensors="pt").to(device)
outputs = model.generate(**inputs)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print(answer[0])

E


In [53]:
submission = pd.read_csv('./sample_submission.csv', index_col='id')

i = 0
for text in input:
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    submission.loc[i,'prediction'] = answer[0]
    i = i+1


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [54]:
submission.to_csv('submission.csv')